In [5]:
import os, sys, pickle
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

# tf and keras
import keras, tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.sequence import pad_sequences

# spacy
import spacy
import en_core_web_sm

### Initialize necessary variables

In [4]:
MAX_LEN = 300

### Test input

In [32]:
test_direction = '''
preheat oven to 350 degrees f (175 degrees c). lightly grease and flour one 9x13 inch cake pan.**
beat vegetable oil and eggs until foamy. add the sugar, flour, ground cinnamon, baking, soda, salt, 
and vanilla and mix well. stir in the diced apples. pour batter into prepared pan.**bake at 350 degrees f 
(175 degrees c) for 30 to 40 minutes. cool cake in pan for 10 minutes. cake needs no frosting.**
'''.strip()

test_ingredients = 'egg,vegetable oil,white sugar,flour,cinnamon,baking soda,salt,vanilla,apple'

### Load model and vocab

In [36]:
# load model

model_path = "../models/fs_lstm_v1.0.h5"
model = keras.models.load_model(model_path)
vocab = pickle.load(open("../models/vocab_v1.0.pkl", "rb"))
# tokens = pickle.load(open("../models/tokens_v1.0.pkl", "rb"))

### Prepare test input

In [37]:
# process sequence from test input

def process_sequences(tokens, max_len, vocab={"<UNK>": 1, "<PAD>": 0}):
    X = [[vocab.get(w.text, vocab["<UNK>"]) for w in s] for s in tokens]
    return pad_sequences(maxlen=max_len, sequences=X, padding="post", value=vocab["<PAD>"])


nlp = en_core_web_sm.load(disable=['parser', 'tagger', 'ner'])
test_tokens = [nlp(test_direction.strip())]

X_test = process_sequences(test_tokens, max_len=MAX_LEN, vocab=vocab)
print(X_test.shape)

(1, 300)


### Predict test input

In [39]:
y_pred = model.predict(X_test, verbose=1)[0]
y_pred = y_pred > 0.05
ingreds = [t.text for t, p in zip(test_tokens[0], y_pred) if p]
ingreds = [i for i in ingreds if i != "\n"]
ingreds = set(ingreds)
ingreds

1/1 [==============================] - 0s 62ms/step


{'apples',
 'baking',
 'cake',
 'cinnamon',
 'eggs',
 'flour',
 'ground',
 'oil',
 'salt',
 'soda',
 'sugar',
 'vanilla',
 'vegetable'}

In [28]:
test_tokens

[
 preheat oven to 350 degrees f (175 degrees c). lightly grease and flour one 9x13 inch cake pan.**
 beat vegetable oil and eggs until foamy. add the sugar, flour, ground cinnamon, baking, soda, salt, 
 and vanilla and mix well. stir in the diced apples. pour batter into prepared pan.**bake at 350 degrees f 
 (175 degrees c) for 30 to 40 minutes. cool cake in pan for 10 minutes. cake needs no frosting.**]